# Locality Sensitive Hashing - Python3 Example

The following Python3 Jupyter Notebook walks through a general example of a LSH example.

### References

* stack overflow example of the lsh where much of the code is from. [How to understand Locality Sensitive Hashing](https://stackoverflow.com/questions/12952729/how-to-understand-locality-sensitive-hashing)
* [Mining of Massive Datasets. Check Chapter 3 - Finding Similar Items](http://infolab.stanford.edu/~ullman/mmds/ch3a.pdf)
* [Locality-Sensitive Hashing for Finding Nearest Neighbors](http://www.slaney.org/malcolm/yahoo/Slaney2008-LSHTutorial.pdf)

In [34]:
import numpy as np
import math

## Signature Generator
LSH signature generation using random projection.  Creation example:

**r1 =** 5258938758107597568730090422086682111341336555356353764484021991907916884238862457752925253205886540063228859967989877341517
3708958214185378191229172581078591707136483695662719666201194820344998416019553425368626333566280373218005140028455336616252
113734411167457082017629313236890406507495424505529953875716 

**r2 =**
4188726764665893952181551897213197536490322759612234627111192449758262888603806182273761430857902365465000582299715182381794
6025553602019495096619768555900041945849259355914615371488201147608435031471657292041543981419502219834960094881943140319526
668290047547998830850975676427533817413288633885065017275967

Where 
* user_vector = is a random dimension (integer) - an array of dim floats
* rand_proj = is number between d and user_vector.  d is number of bits per signature.
* x << y = Returns x with the bits shifted to the left by y places (and new bits on the right-hand-side are zeros). This is the same as multiplying x by 2**y.

**Output:**
* res = number of 1's in dot product of rand_proj and user_vector

In [35]:
def get_signature(user_vector, rand_proj): 
    res = 0
    val = np.dot(rand_proj, user_vector)
    for v in val:
        res = res << 1
        if v >= 0:
            res += 1
    return res

# Binary Tally

Where:
* num = xor = r1^r2
* x & y = Does a "bitwise and". Each bit of the output is 1 if the corresponding bit of x AND of y is 1, otherwise it's 0.

In [36]:
# get number of '1's in binary
# running time: O(# of '1's)
def nnz(num):
    if num == 0:
        return 0
    res = 1
    num = num & (num-1)
    while num:
        res += 1
        num = num & (num-1)
    return res     

## Cosine Similarity

Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space. It is defined to equal the cosine of the angle between them, which is also the same as the inner product of the same vectors normalized to both have length 1. The cosine of 0° is 1, and it is less than 1 for any angle in the interval (0, π] radians. It is thus a judgment of orientation and not magnitude: two vectors with the same orientation have a cosine similarity of 1, two vectors oriented at 90° relative to each other have a similarity of 0, and two vectors diametrically opposed have a similarity of -1, independent of their magnitude. The cosine similarity is particularly used in positive space, where the outcome is neatly bounded in [0,1].

In [37]:
# angular similarity using definitions
# http://en.wikipedia.org/wiki/Cosine_similarity
def angular_similarity(a,b):
    dot_prod = np.dot(a,b)
    sum_a = sum(a**2) **.5
    sum_b = sum(b**2) **.5
    cosine = dot_prod/sum_a/sum_b # cosine similarity
    theta = math.acos(cosine)
    return 1.0-(theta/math.pi)

# Main

User defines:
* dim - number of dimensions per data
* d - number of bits per signature
* nruns - repeat times

**Outputs:**
* true_sim = cosine similarity meassure between user_1 and user_2
* hash_sim = binary tally
* diff = percent difference

**Example:**


In [38]:
if __name__ == '__main__':
    dim = 200 # number of dimensions per data
    d = 2**4 # number of bits per signature
    
    nruns = 24 # repeat times
    
    avg = 0
    for run in range(nruns):
        user1 = np.random.randn(dim)
        user2 = np.random.randn(dim)
        randv = np.random.randn(d, dim)    
        r1 = get_signature(user1, randv)
        r2 = get_signature(user2, randv)
        xor = r1^r2
        true_sim, hash_sim = (angular_similarity(user1, user2), (d-nnz(xor))/float(d))
        diff = abs(hash_sim-true_sim)/true_sim
        avg += diff
        print('true %.4f, hash %.4f, diff %.4f' % (true_sim, hash_sim, diff)) 
    print('avg diff' , avg / nruns)

true 0.4852, hash 0.3750, diff 0.2271
true 0.4802, hash 0.4375, diff 0.0890
true 0.4786, hash 0.3750, diff 0.2164
true 0.4350, hash 0.3750, diff 0.1379
true 0.5044, hash 0.5000, diff 0.0086
true 0.4521, hash 0.4375, diff 0.0323
true 0.4941, hash 0.4375, diff 0.1145
true 0.5210, hash 0.3125, diff 0.4002
true 0.4558, hash 0.3750, diff 0.1772
true 0.5225, hash 0.6875, diff 0.3159
true 0.4834, hash 0.4375, diff 0.0949
true 0.5098, hash 0.5625, diff 0.1033
true 0.5291, hash 0.4375, diff 0.1732
true 0.5088, hash 0.5000, diff 0.0174
true 0.5086, hash 0.3750, diff 0.2627
true 0.5182, hash 0.5000, diff 0.0351
true 0.4672, hash 0.5000, diff 0.0702
true 0.5151, hash 0.5000, diff 0.0293
true 0.5271, hash 0.6250, diff 0.1858
true 0.4797, hash 0.6250, diff 0.3028
true 0.5176, hash 0.6250, diff 0.2074
true 0.4909, hash 0.6875, diff 0.4005
true 0.5261, hash 0.5625, diff 0.0693
true 0.5140, hash 0.5000, diff 0.0272
avg diff 0.1541026093525961
